In [14]:
import os

import pandas as pd

In [15]:
path = os.getcwd()
df = pd.read_json(path + '/archive/MMHS150K_GT.json', orient='index')
df.sample(5)

,img_url,labels,tweet_url,tweet_text,labels_str
2005-05-21 00:15:17.828034561,http://pbs.twimg.com/media/D38VLEMUIAAKFeP.jpg,"[0, 1, 0]",https://twitter.com/user/status/11166345178280...,I’m sorry but my nigga Jungkook https://t.co/X...,"[NotHate, Racist, NotHate]"
2003-01-11 15:08:31.519776768,http://pbs.twimg.com/media/Dnb8PCdXsAAfPR6.jpg,"[0, 0, 0]",https://twitter.com/user/status/10422977115197...,Column van Hans Rube #Tilburg Hengelsport http...,"[NotHate, NotHate, NotHate]"
2005-01-31 02:22:09.549553665,http://pbs.twimg.com/tweet_video_thumb/D11YQGy...,"[1, 0, 1]",https://twitter.com/user/status/11071381295495...,Me and the one other black person in the club ...,"[Racist, NotHate, Racist]"
2005-02-07 20:33:49.878337536,http://pbs.twimg.com/media/D1-54xIUwAA-2eX.jpg,"[1, 0, 3]",https://twitter.com/user/status/11078084298783...,Sissy Faggot Plugged.... https://t.co/1WfillkiCd,"[Racist, NotHate, Homophobe]"
2005-05-11 15:53:15.461402624,http://pbs.twimg.com/ext_tw_video_thumb/111582...,"[0, 0, 1]",https://twitter.com/user/status/11158267954614...,This nigga Harden is the WORST 😭 https://t.co/...,"[NotHate, NotHate, Racist]"


In [16]:
# labels: 0 - NotHate, 1 - Racist, 2 - Sexist, 3 - Homophobe, 4 - Religion, 5 - OtherHate

# extract tweet id from tweet url
df['tweet_id'] = df['tweet_url'].apply(lambda x: x.split('/')[-1])

# create a new column for each label
df['is_hateful'] = df['labels'].apply(lambda x: x !=[0,0,0])
# create a new column for each label
df['is_racist'] = df['labels'].apply(lambda x: 1 in x)
df['is_sexist'] = df['labels'].apply(lambda x: 2 in x)
df['is_homophobe'] = df['labels'].apply(lambda x: 3 in x)
df['is_religion'] = df['labels'].apply(lambda x: 4 in x)
df['is_other_hate'] = df['labels'].apply(lambda x: 5 in x)

# remove the labels column
df = df.drop(columns=['labels'])
# remove the labels_str column
df = df.drop(columns=['labels_str'])

# reset index => remove date and get a new index
df = df.reset_index(drop=True)

print(df.is_hateful.value_counts())
df.head()

is_hateful
True     91933
False    57890
Name: count, dtype: int64


,img_url,tweet_url,tweet_text,tweet_id,is_hateful,is_racist,is_sexist,is_homophobe,is_religion,is_other_hate
0,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,1114679353714016256,True,True,False,True,True,False
1,http://pbs.twimg.com/ext_tw_video_thumb/106301...,https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,1063020048816660480,True,False,False,False,False,True
2,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,1108927368075374593,False,False,False,False,False,False
3,http://pbs.twimg.com/ext_tw_video_thumb/111401...,https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,1114558534635618305,True,True,False,False,False,False
4,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,1035252480215592966,True,True,False,False,False,False


In [17]:
# Now I want to create a subset of the data with equal number of hateful and non-hateful tweets
# for hateful, to make it easier, we will consider only the tweets multiple labels 
# (i.e., n_labels > 1)
df_hateful = df[df['is_hateful'] == True]
df_non_hateful = df[df['is_hateful'] == False]

min_data = 10000 + 1000 # for inference dataset
num_of_each_class = min(len(df_hateful), len(df_non_hateful), min_data)
print("Number of sample in each class:", num_of_each_class)

df_hateful = df_hateful.sample(n=num_of_each_class, random_state=42)
df_non_hateful = df_non_hateful.sample(n=num_of_each_class, random_state=42)

df_sample = pd.concat([df_hateful, df_non_hateful], axis=0)

df_sample = df_sample.drop(columns=['is_racist', 'is_sexist', 'is_homophobe', 'is_religion', 'is_other_hate'])

df_training = df_sample.sample(frac=0.9, random_state=42)
df_inference = df_sample.drop(df_training.index)

# Save the data for later use 
# Data for training and testing the model
df_training.to_csv(path + '/data/MMHS150K_GT.csv', index=False)

# Data for inference
df_inference.to_csv(path + '/data/MMHS150K_GT_inference.csv', index=False)

df_sample.head()

Number of sample in each class: 11000


,img_url,tweet_url,tweet_text,tweet_id,is_hateful
35353,http://pbs.twimg.com/media/D1hhczLWkAI8q7f.jpg,https://twitter.com/user/status/11057408704632...,@iveylee091200 @nowthisnews Not without a figh...,1105740870463225856,True
108931,http://pbs.twimg.com/tweet_video_thumb/DqcrO0Q...,https://twitter.com/user/status/10558601851708...,@Iainsh @NadsLFC @WiggzLFC @_TheBoss Lieing tw...,1055860185170886657,True
16627,http://pbs.twimg.com/media/D3z9oe-XoAEtOzo.jpg,https://twitter.com/user/status/11160456857434...,HIT A SWITCH ON A FAKE NIGGA LIKE A STATION ht...,1116045685743411205,True
103737,http://pbs.twimg.com/ext_tw_video_thumb/106172...,https://twitter.com/user/status/10627646069496...,@ChinkyyChink Me acting like i wouldn't let my...,1062764606949613570,True
130333,http://pbs.twimg.com/media/DoS5pJvUgAA-7n0.jpg,https://twitter.com/user/status/10461651356504...,a whole dyke mood 😂 #LGBTQTwiter https://t.co/...,1046165135650410501,True


In [18]:
# we need to restrure the image folder
# to be able to use the torchvision.transforms
# not_hateful images need to be in the folder not_hateful
# hateful images need to be in the folder hateful
from os.path import join
import os
import shutil

# create the folders
os.makedirs("data/images/not_hateful", exist_ok=True)
os.makedirs("data/images/hateful", exist_ok=True)

# clear data folders
shutil.rmtree("data/images/not_hateful")
shutil.rmtree("data/images/hateful")

# create the folders
os.makedirs("data/images/not_hateful", exist_ok=True)
os.makedirs("data/images/hateful", exist_ok=True)

# for image in images:
for image in df_sample['tweet_id']:
    if not os.path.exists(join("archive/img_resized", f"{image}.jpg")):
        print(f"Image {image} not found")
        continue
    # if the image is hateful
    if df_sample[df_sample['tweet_id'] == image]['is_hateful'].values[0]:
        # move the image to the hateful folder
        source = join("archive/img_resized", f"{image}.jpg")
        destination = join("data/images/hateful", f"{image}.jpg")
    else:
        # move the image to the not hateful folder
        source = join("archive/img_resized", f"{image}.jpg")
        destination = join("data/images/not_hateful", f"{image}.jpg")
                
    shutil.copy(source, destination)